<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/015_Feedback_Agent_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## 🧠 Feedback Sentiment Analysis Agent

**Purpose:**
This notebook extends our existing Feedback Intelligence Agent by introducing **sentiment analysis** to better understand the emotional tone behind employee feedback at Target.

### 🎯 Goal

Identify and track how employees feel about their work environment — not just what they say, but **how they say it**.

### ✅ What This Agent Will Do

* Analyze employee feedback text for **sentiment** (Positive / Neutral / Negative)
* Combine this sentiment data with existing:

  * Department (e.g., GM, OOF)
  * Topic (e.g., scanner issues, cart availability)
  * Summary (from our previous agent)
* Surface:

  * Which departments show more frustration or positivity
  * Which topics are emotionally charged
  * Trends or pain points worth investigating further

### 🔍 Why This Matters

* **Negative feedback** often signals operational friction, burnout, or poor training
* **Positive sentiment** can highlight what’s working — and where leadership is succeeding
* **Neutral feedback** often reflects informative but unemotional reporting

Understanding **tone + topic together** gives store managers a more complete, empathetic view of the employee experience.




##Pip Installs

In [ ]:
!pip install -q transformers accelerate huggingface_hub litellm python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.1 MB/s eta 0:00:00


## Feedback Samples

In [ ]:
import json

# Sample feedback data
feedback_data = [
    # GM: Merchandising
    {
        "department": "GM",
        "employee_feedback": "The shelves in aisle 12 are constantly messy because customers leave things everywhere and we don’t have time to fix it between tasks."
    },
    {
        "department": "GM",
        "employee_feedback": "Our pricing gun breaks down frequently. It slows down the whole price change process."
    },
    {
        "department": "GM",
        "employee_feedback": "We never have enough shopping carts near the entrance during busy hours."
    },
    {
        "department": "GM",
        "employee_feedback": "The planogram we receive doesn’t match the actual product layout — makes resets take longer."
    },
    {
        "department": "GM",
        "employee_feedback": "Too many tasks assigned during a single shift. We need more hands during truck unload days."
    },

    # OOF: Online Order Fulfillment
    {
        "department": "OOF",
        "employee_feedback": "Pick times are too short for some items that are far apart — leads to constant rushing."
    },
    {
        "department": "OOF",
        "employee_feedback": "Sometimes online orders include out-of-stock items — makes us look bad and causes delays."
    },
    {
        "department": "OOF",
        "employee_feedback": "The handheld scanner freezes mid-pick and we have to restart it almost daily."
    },
    {
        "department": "OOF",
        "employee_feedback": "We aren’t told when substitutes are approved — we waste time checking back."
    },
    {
        "department": "OOF",
        "employee_feedback": "There’s not enough space in the staging area when online orders pile up during weekends."
    },
    {
        "department": "GM",
        "employee_feedback": "Customers constantly leave items in the wrong places and we don’t have time to fix it all during shift changes."
    },
    {
        "department": "GM",
        "employee_feedback": "We barely have enough carts in the front during rush hours — customers get annoyed."
    },
    {
        "department": "GM",
        "employee_feedback": "It’s frustrating when planograms are wrong or outdated — we waste a lot of time fixing shelves that shouldn't need changing."
    },
    {
        "department": "OOF",
        "employee_feedback": "I often can’t find items in time because they're far apart, and the pick times are too short."
    },
    {
        "department": "OOF",
        "employee_feedback": "The scanner crashes a lot — restarting it wastes at least 10 minutes each time."
    }
]

# Save to JSON
with open("sample_feedback.json", "w") as f:
    json.dump(feedback_data, f, indent=2)

print("✅ Sample feedback saved to sample_feedback.json")


✅ Sample feedback saved to sample_feedback.json


## Load API Key

In [ ]:
import json
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv("/content/API_KEYS.env", override=True)

# Grab API key
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("❌ OPENAI_API_KEY not found in environment. Make sure your .env file is loaded correctly.")

# Set up OpenAI client
client = OpenAI(api_key=api_key)

# Load sample feedback
with open("sample_feedback.json", "r") as f:
    feedback_list = json.load(f)

print(f"✅ Loaded {len(feedback_list)} feedback entries")

✅ Loaded 15 feedback entries


## Feedback Function

## ✅ Can One Model Handle Both Summary *and* Sentiment?

**Short answer: YES — if prompted correctly.**
You can absolutely ask a single OpenAI model (like `gpt-4o-mini`) to:

1. Summarize the feedback
2. Generate a topic tag
3. Classify sentiment

...**all in one structured JSON response.**

In fact, you're already halfway there — these are basic reasoning tasks that GPT-4/4o handles well in a single prompt.

---

### ✅ When a Single Model/Call Is Best

Use a single function if:

| ✅ You want to...                       | Because...                       |
| -------------------------------------- | -------------------------------- |
| Reduce API costs                       | One call instead of two          |
| Keep latency low                       | Faster pipelines                 |
| Keep logic in one place                | Simpler codebase                 |
| Use one prompt to control all behavior | Better control over tone/quality |

💡 **You’re already doing this** in `analyze_feedback()` — just extend the prompt slightly.

---

### ❌ When You *Might* Want to Separate Them

You’d split into 2 steps only if:

| ❌ You need to...                      | Why...                                                   |
| ------------------------------------- | -------------------------------------------------------- |
| Use a non-LLM sentiment model         | e.g., fine-tuned BERT, Vader, etc.                       |
| Change models for performance         | Use GPT-4 for summary, GPT-3.5 for sentiment             |
| Treat sentiment as its own pipeline   | For feedback dashboards, survey-style scoring            |
| Use advanced sentiment classification | (e.g., mixed sentiment, emotion tags, sarcasm detection) |

For your case — simple 3-level sentiment (Positive, Neutral, Negative) — **LLM is sufficient and accurate.**

---

## ✅ What Model Should You Use?

| Model           | Good for...               | Why it's a fit                              |
| --------------- | ------------------------- | ------------------------------------------- |
| `gpt-4o-mini`   | Summary, topic, sentiment | Cheap, fast, accurate at structured output  |
| `gpt-4o`        | Slightly better reasoning | Overkill for your current task              |
| `gpt-3.5-turbo` | Adequate for most tasks   | Cheaper, but sometimes worse at strict JSON |

You're already using `gpt-4o-mini`, and it’s the **right call** for this job.






In [ ]:
def analyze_feedback(feedback_text, department):
    system_prompt = f"""
You are an assistant that helps summarize and categorize employee feedback from a retail store.

The employee works in the '{department}' department.

For each piece of feedback, do the following:
1. Summarize it in one clear sentence.
2. Assign a concise topic label (e.g., "scanner issues", "cart availability").
3. Classify the sentiment as one of: "Positive", "Neutral", or "Negative".

Use the following format:
{{
  "summary": "...",
  "topic": "...",
  "sentiment": "Positive | Neutral | Negative"
}}

Here are some examples:

---

Employee: "The handheld scanner freezes mid-pick and we have to restart it almost daily."
Output:
{{
  "summary": "Scanners freeze frequently, causing workflow delays.",
  "topic": "scanner issues",
  "sentiment": "Negative"
}}

---

Employee: "The new cart system works really well — customers don’t have to wait for carts anymore."
Output:
{{
  "summary": "Cart availability has improved and customer experience is better.",
  "topic": "cart availability",
  "sentiment": "Positive"
}}

---

Employee: "Sometimes the planogram doesn’t match the shelf layout, which slows us down."
Output:
{{
  "summary": "Mismatch between planogram and shelves causes delays.",
  "topic": "planogram mismatch",
  "sentiment": "Neutral"
}}

---

Now analyze this feedback:
"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": feedback_text}
    ]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.3
    )

    reply = response.choices[0].message.content.strip()

    try:
        result = json.loads(reply)
        result["department"] = department
        return result
    except json.JSONDecodeError:
        print("❌ Failed to parse response:", reply)
        return {
            "summary": "Could not parse",
            "topic": "uncategorized",
            "sentiment": "Neutral",
            "department": department
        }



## Process All Feedback and Store Results





In [ ]:
import pandas as pd

results = []

for entry in feedback_list:
    dept = entry["department"]
    text = entry["employee_feedback"]

    print(f"🔍 Processing ({dept}): {text}")
    result = analyze_feedback(text, dept)

    results.append({
        "department": result["department"],
        "original_feedback": text,
        "summary": result["summary"],
        "topic": result["topic"],
        "sentiment": result["sentiment"]  # ✅ New line
    })

# Convert results to a DataFrame
df = pd.DataFrame(results)
df.head()


🔍 Processing (GM): The shelves in aisle 12 are constantly messy because customers leave things everywhere and we don’t have time to fix it between tasks.
🔍 Processing (GM): Our pricing gun breaks down frequently. It slows down the whole price change process.
🔍 Processing (GM): We never have enough shopping carts near the entrance during busy hours.
🔍 Processing (GM): The planogram we receive doesn’t match the actual product layout — makes resets take longer.
🔍 Processing (GM): Too many tasks assigned during a single shift. We need more hands during truck unload days.
🔍 Processing (OOF): Pick times are too short for some items that are far apart — leads to constant rushing.
🔍 Processing (OOF): Sometimes online orders include out-of-stock items — makes us look bad and causes delays.
🔍 Processing (OOF): The handheld scanner freezes mid-pick and we have to restart it almost daily.
🔍 Processing (OOF): We aren’t told when substitutes are approved — we waste time checking back.
🔍 Processing (

,department,original_feedback,summary,topic,sentiment
0,GM,The shelves in aisle 12 are constantly messy b...,Aisle 12 shelves remain messy due to customer ...,shelf organization,Negative
1,GM,Our pricing gun breaks down frequently. It slo...,Frequent breakdowns of the pricing gun hinder ...,pricing gun issues,Negative
2,GM,We never have enough shopping carts near the e...,There is a shortage of shopping carts near the...,cart availability,Negative
3,GM,The planogram we receive doesn’t match the act...,The received planogram does not align with the...,planogram mismatch,Negative
4,GM,Too many tasks assigned during a single shift....,Excessive tasks during shifts necessitate addi...,staffing needs,Negative


## Review Sentiment Results

In [ ]:
# Filter only negative feedback
negative_feedback = df[df["sentiment"] == "Negative"]

# Print each entry in full
for i, row in negative_feedback.iterrows():
    print("🔻 Negative Feedback")
    print(f"📝 Original: {row['original_feedback']}")
    print(f"🧠 Summary : {row['summary']}")
    print(f"📌 Topic   : {row['topic']}")
    print("-" * 60)


🔻 Negative Feedback
📝 Original: The shelves in aisle 12 are constantly messy because customers leave things everywhere and we don’t have time to fix it between tasks.
🧠 Summary : Aisle 12 shelves remain messy due to customer disorganization and limited staff time for restocking.
📌 Topic   : shelf organization
------------------------------------------------------------
🔻 Negative Feedback
📝 Original: Our pricing gun breaks down frequently. It slows down the whole price change process.
🧠 Summary : Frequent breakdowns of the pricing gun hinder the price change process.
📌 Topic   : pricing gun issues
------------------------------------------------------------
🔻 Negative Feedback
📝 Original: We never have enough shopping carts near the entrance during busy hours.
🧠 Summary : There is a shortage of shopping carts near the entrance during peak times.
📌 Topic   : cart availability
------------------------------------------------------------
🔻 Negative Feedback
📝 Original: The planogram we re



### ✅ **What’s Working Well**

#### 🔹 Accurate Negative Classification

All 15 examples you posted are **correctly tagged as "Negative"**, and here’s why that’s important:

* Every entry describes **frustration, disruption, or inefficiency**.
* The summaries are well-worded and precise.
* The topics are meaningful, repeatable, and suitable for grouping.

#### 🔹 Solid Topic Tagging

Your topics like `"scanner issues"`, `"planogram mismatch"`, and `"cart availability"` are:

* **Concise but clear**
* **Consistently phrased** — making them perfect for grouping
* ✅ Useful in a dashboard, chart, or report

#### 🔹 Summaries Preserve Intent

The summaries reduce the noise and keep the **employee’s original concern intact**, e.g.:

> 📝 *"Customers constantly leave items in the wrong places and we don’t have time to fix it all..."*
> 🧠 *"Customers frequently leave items in incorrect locations, making it difficult to manage during shift changes."*

That’s exactly the kind of fidelity you want from summarization.

---

### 🔍 Suggestions (Very Minor)

#### 1. 🧹 Topic Normalization (optional)

You might eventually want to **standardize overlapping topics**:

| Current Label                                   | Suggested Normalization |
| ----------------------------------------------- | ----------------------- |
| `planogram mismatch` / `planogram issues`       | → `planogram issues`    |
| `item location and pick times` / `pick times`   | → `pick time issues`    |
| `cart availability` / `carts in front` (future) | → `cart availability`   |

→ You could do this post-processing manually or with a rules-based mapping.

#### 2. 🧪 Add a few positive/neutral entries for coverage

To validate that the model is **not over-classifying everything as negative**, try testing a few:

* Positive feedback (e.g., “New scanner model is faster!”)
* Truly neutral statements (e.g., “The layout was recently updated.”)

That will help you confirm **sentiment balance**.

---

### 🧠 TL;DR – Feedback Review Summary

| Category                | Rating      | Notes                                                   |
| ----------------------- | ----------- | ------------------------------------------------------- |
| Sentiment accuracy      | ✅ Excellent | All examples correctly classified as negative           |
| Summary precision       | ✅ Strong    | Preserves tone + key concern clearly                    |
| Topic consistency       | ✅ Very Good | Slight overlaps, but well-labeled and actionable        |
| Readiness for next step | ✅ Ready     | You can now group + score by topic/department/sentiment |



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set plot style
sns.set(style="whitegrid")

# Create a countplot of topic frequency
plt.figure(figsize=(12, 6))
sns.countplot(data=df, y="topic", order=df["topic"].value_counts().index)

plt.title("Frequency of Topics Mentioned in Employee Feedback")
plt.xlabel("Number of Mentions")
plt.ylabel("Topic")
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
sns.countplot(data=df, y="topic", hue="department", order=df["topic"].value_counts().index, palette='Set1_r')

plt.title("Topic Mentions by Department")
plt.xlabel("Number of Mentions")
plt.ylabel("Topic")
plt.legend(title="Department")
plt.tight_layout()
plt.show()


## Create Topic Normilzation Map

In [ ]:
# Define mapping from messy/raw topics to clean, standardized ones
topic_normalization_map = {
    "planogram mismatch": "planogram issues",
    "planogram issues": "planogram issues",
    "pick times": "pick time issues",
    "item location and pick times": "pick time issues",
    "cart availability": "cart availability",
    "carts in front": "cart availability",
    "scanner issues": "scanner issues",  # Already standardized
    "pricing gun issues": "equipment issues",
    "shelf organization": "shelf maintenance",
    "item placement": "shelf maintenance",
    "staging area space": "storage limitations",
    "staffing needs": "staffing",
    "communication issues": "communication issues",
    "out-of-stock items": "inventory problems"
}

# Normalize topics in the DataFrame
df["normalized_topic"] = df["topic"].map(topic_normalization_map).fillna(df["topic"])
df.head()

,department,original_feedback,summary,topic,sentiment,normalized_topic
0,GM,The shelves in aisle 12 are constantly messy b...,Aisle 12 shelves remain messy due to customer ...,shelf organization,Negative,shelf maintenance
1,GM,Our pricing gun breaks down frequently. It slo...,Frequent breakdowns of the pricing gun hinder ...,pricing gun issues,Negative,equipment issues
2,GM,We never have enough shopping carts near the e...,There is a shortage of shopping carts near the...,cart availability,Negative,cart availability
3,GM,The planogram we receive doesn’t match the act...,The received planogram does not align with the...,planogram mismatch,Negative,planogram issues
4,GM,Too many tasks assigned during a single shift....,Excessive tasks during shifts necessitate addi...,staffing needs,Negative,staffing
